In [19]:
# Code pulled from SO, https://stackoverflow.com/a/45036114/665578

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
import scipy.stats as scs
import statsmodels.api as sm

%matplotlib inline

In [2]:
# https://stackoverflow.com/questions/45021301/geometric-brownian-motion-simulation-in-python
    
def gen_paths(S0, r, sigma, T, M, I):
    """Generate Monte Carlo paths for geometric Brownian motion.
    
    https://github.com/yhilpisch/py4fi/blob/master/jupyter36/11_Statistics_a.ipynb

    Args:
        S0 (float): initial stock price
        r (float): constant short rate
        sigma (float): constant volatility
        T (float): final time horizon
        M (int): number of time/step intervals
        I (int): number of paths to simulate

    Returns:
        nd.array, shape(M+1, I): simulated paths
    """
    dt = float(T) / M
    paths = np.zeros((M + 1, I), np.float64)
    paths[0] = S0
    for t in range(1, M + 1):
        rand = np.random.standard_normal(I)
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * rand)
    return paths

def print_statistics(array):
    ''' Prints selected statistics.
    
    Parameters
    ==========
    array: ndarray
        object to generate statistics on
    '''
    sta = scs.describe(array)
    print("%14s %15s" % ('statistic', 'value'))
    print(30 * "-")
    print("%14s %15.5f" % ('size', sta[0]))
    print("%14s %15.5f" % ('min', sta[1][0]))
    print("%14s %15.5f" % ('max', sta[1][1]))
    print("%14s %15.5f" % ('mean', sta[2]))
    print("%14s %15.5f" % ('std', np.sqrt(sta[3])))
    print("%14s %15.5f" % ('skew', sta[4]))
    print("%14s %15.5f" % ('kurtosis', sta[5]))


class Option(object):
    """
    Compute European option value, greeks, and implied volatility.

    Parameters
    ==========
    S0 : int or float
        initial asset value
    K : int or float
        strike
    T : int or float
        time to expiration as a fraction of one year
    r : int or float
        continuously compounded risk free rate, annualized
    sigma : int or float
        continuously compounded standard deviation of returns
    kind : str, {'call', 'put'}, default 'call'
        type of option

    Resources
    =========
    http://www.thomasho.com/mainpages/?download=&act=model&file=256
    """
    def __init__(self, S0, K, T, r, sigma, kind='call'):
        if kind.istitle():
            kind = kind.lower()
        if kind not in ['call', 'put']:
            raise ValueError('Option type must be \'call\' or \'put\'')

        self.kind = kind
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma

        self.d1 = ((np.log(self.S0 / self.K)
                + (self.r + 0.5 * self.sigma ** 2) * self.T)
                / (self.sigma * np.sqrt(self.T)))
        self.d2 = ((np.log(self.S0 / self.K)
                + (self.r - 0.5 * self.sigma ** 2) * self.T)
                / (self.sigma * np.sqrt(self.T)))

        # Several greeks use negated terms dependent on option type
        # For example, delta of call is N(d1) and delta put is N(d1) - 1
        self.sub = {'call' : [0, 1, -1], 'put' : [-1, -1, 1]}

    def value(self):
        """Compute option value."""
        return (self.sub[self.kind][1] * self.S0
               * norm.cdf(self.sub[self.kind][1] * self.d1, 0.0, 1.0)
               + self.sub[self.kind][2] * self.K * np.exp(-self.r * self.T)
               * norm.cdf(self.sub[self.kind][1] * self.d2, 0.0, 1.0))

In [8]:
np.random.seed(123)

S0 = 72.
K = 72.
r = 0.01
sigma = 0.28
T = 1.0
N = 30
deltat = T / N
i = 10000

discount_factor = np.exp(-r * T)
paths = gen_paths(S0, r, sigma, T, N, i)

log_returns = np.log(paths[1:] / paths[0:-1]) 
print_statistics(log_returns.flatten())

if i < 100:
    pd.DataFrame(paths).plot()
else:
    print("skipping plot since i > 100")

     statistic           value
------------------------------
          size    300000.00000
           min        -0.23649
           max         0.22140
          mean        -0.00087
           std         0.05104
          skew         0.00168
      kurtosis        -0.00174
skipping plot since i > 100


In [4]:
S0 = 100.0
K = 100.0
r = 0.05
sigma = 0.29
T = 1.0
N = 252
deltat = T / N
i = 10

option = Option(S0, K, T, r, sigma, kind='call')
option.value()

print(f"Black-Scholes value, call = {option.value():.2f}")


for index in range(990, 1000):
    i = index * 100
    discount_factor = np.exp(-r * T)
    np.random.seed(123)
    paths = gen_paths(S0, r, sigma, T, N, i)

    CallPayoffAverage = np.average(np.maximum(0, paths[-1] - K))
    CallPayoff = discount_factor * CallPayoffAverage
    print(f"Monte-Carlo simulated CallPayoff = {CallPayoff:.2f}")

Black-Scholes value, call = 13.85
Monte-Carlo simulated CallPayoff = 13.84
Monte-Carlo simulated CallPayoff = 13.94
Monte-Carlo simulated CallPayoff = 13.86
Monte-Carlo simulated CallPayoff = 13.89
Monte-Carlo simulated CallPayoff = 13.84
Monte-Carlo simulated CallPayoff = 13.93
Monte-Carlo simulated CallPayoff = 13.92
Monte-Carlo simulated CallPayoff = 13.93
Monte-Carlo simulated CallPayoff = 13.90
Monte-Carlo simulated CallPayoff = 13.86
